In [2]:
class Node:
 def __init__(self, left_child_index):
  self.DataValue = ""
  self.Expression = []
  self.LeftChild = left_child_index
  self.RightChild = -1

In [72]:
class ExpressionTree:

    def __init__(self):
        self.Tree = list()
        for index in range(1, 21):
            self.Tree.append(Node(index))
            self.Fringe = list()
            self.Root = 0
            self.NextFreeChild = 0

    def IsOperator(self, s):
        if '+' in s:
            return True
        if '-' in s:
            return True
        if '*' in s:
            return True
        if '/' in s:
            return True
        return False
    
    def Insert(self, NewToken):
        if self.NextFreeChild == -1: # check if tree is full
            return "Tree is Full"

        # tree is not full, safe to insert new token
        if self.NextFreeChild == 0:
            self.Tree[self.Root].DataValue = NewToken
            self.NextFreeChild = self.Tree[self.Root].LeftChild
            self.Tree[self.Root].LeftChild = -1
        else:
            # insert into tree with existing nodes
            # starting with Root
            Current = 0 # index of the current node
            Previous = -1 # index of previous node
            NewNode = self.Tree[self.NextFreeChild] # declare new node
            NewNode.DataValue = NewToken

            # Finding the node at which the NewNode can be inserted
            while Current != -1:
                CurrNode = self.Tree[Current]
                # check if CurrNode contains an operator
                if self.IsOperator(CurrNode.DataValue):
                # if LeftChild is empty, insert here
                    if CurrNode.LeftChild == -1:
                        CurrNode.LeftChild = self.NextFreeChild
                        self.NextFreeChild = NewNode.LeftChild
                        NewNode.LeftChild = -1
                        Current = -1
                        # if RightChild is empty, insert here
                    elif CurrNode.RightChild == -1:
                        CurrNode.RightChild = self.NextFreeChild
                        self.NextFreeChild = NewNode.LeftChild
                        NewNode.LeftChild = -1
                        Current = -1
                    # if LeftChild is an operator
                    # traverse LeftChild subtree
                    elif self.IsOperator(self.Tree[CurrNode.LeftChild].DataValue):
                        Previous = Current
                        Current = CurrNode.LeftChild
                        self.Fringe.append(Previous)
                    # if RightChild is an operator
                    # traverse RightChild subtree
                    elif self.IsOperator(self.Tree[CurrNode.RightChild].DataValue):
                        Previous = Current
                        Current = CurrNode.RightChild
                        self.Fringe.append(Previous)
                    # traverse right subtree
                    else:
                        Previous = self.Fringe.pop(-1)
                        Current = self.Tree[Previous].RightChild
                # no place to insert
                else:
                    return "Cannot be inserted"

    def Display(self):
        for index in range(len(self.Tree)):
            print("Index: ", index, "DataValue: ",self.Tree[index].DataValue)
            
    def Infix(self, prefix):
        """
        if root.DataValue != "":
            if self.IsOperator(root.DataValue):
                arr.append('(')
            self.Infix(self.Tree[root.LeftChild], arr)
            arr.append(root.DataValue)
            self.Infix(self.Tree[root.RightChild], arr)
            if self.IsOperator(root.DataValue):
                arr.append(')')
        """
        # read prefix in reverse order
        arr = []
        i = len(prefix) - 1
        while i >= 0:
            if not self.IsOperator(prefix[i]):
                # symbol is operand
                arr.append(prefix[i])
                i -= 1
            else:
                # symbol is operator
                str = "(" + arr.pop() + prefix[i] + arr.pop() + ")"
                arr.append(str)
                i -= 1

        result = arr[0]
        return result[1:len(result)-1]

    def calculate(self, expression):
        def processing(left, stack):
            right = stack.pop() # get right number
            right = int(right)
            operator = stack.pop() # get operator
            left = stack.pop() # get left number
            left = int(left)
            # calculating depending on operator
            if '+' in operator:
                left += right
            elif '-' in operator:
                left -= right
            elif '*' in operator:
                left *= right
            elif '/' in operator:
                left /= right
            return left, stack
        stack = []
        count = 0
        left = 0
        for char in expression:
            stack.append(char)
            if char == ')':
                stack.pop()
                left, stack = processing(left, stack)
                stack.pop()
                stack.append(left)
            if count == len(expression)-1: # last char of expression string
                left, stack = processing(left, stack)
            count += 1
        return left

    def Parser(self, exp):
        #3+56*2
        expression = []
        final = None
        for x in range(0, len(exp)):
            if final is not None and x < final:
                continue

            if self.IsOperator(exp[x]):
                expression.append(exp[x])
            elif x == len(exp) - 1:
                expression.append(exp[x])
            elif self.IsOperator(exp[x+1]) and (self.IsOperator(exp[x-1]) or x==0):
                expression.append(exp[x])
            else:
                multi_dig = exp[x]
                for y in range(x + 1, len(exp)-1):
                    if self.IsOperator(exp[y]) or y == len(exp):
                        final = y
                        break
                    else:
                        multi_dig = multi_dig + exp[y]
                expression.append(multi_dig)
        return expression
        
    def MakeTree(self, exp):
        expression = self.infixToPrefix(self.Parser(exp))
        print(expression)
        Node.Expression = expression
        for item in expression:
            self.Insert(item)
        return self.Display()
        
    """
    def sorter(self, exp):
        sorted_exp = []
        index_pm = None
        index_md = None
        for x in range(0, len(exp)-1):
            if self.IsOperator(exp[x]):
                if exp[x] == '+' or exp[x] == '-':
                    index_pm = x
                    break

        if index_pm:
            sorted_exp.append(exp[index_pm])
            sorted_exp.append(self.sorter(exp[0:index_pm - 1]))
            sorted_exp.append(self.sorter(exp[index_pm+1:]))
        else:
            for x in range(0, len(exp)-1):
                if self.IsOperator(exp[x]):
                    if exp[x] == '*' or exp[x] == '/':
                        index_md = x
                        break
        
        if index_md:
            sorted_exp.append(exp[index_md])
            sorted_exp.append(self.sorter(exp[0:index_md - 1]))
            sorted_exp.append(self.sorter(exp[index_md + 1:]))
        else:
            return exp

        print(sorted_exp)
        return sorted_exp
    """

    def getPriority(self, C):
        if (C == '-' or C == '+'):
            return 1
        elif (C == '*' or C == '/'):
            return 2
        elif (C == '^'):
            return 3
        return 0

    def infixToPrefix(self, infix):
        operators = []
        operands = []
        infix = self.Parser(infix)
        print(infix)
    
        for i in range(len(infix)):
            if (infix[i] == '('):
                operators.append(infix[i])
            elif (infix[i] == ')'):
                while (len(operators)!=0 and operators[-1] != '('):
                    op1 = operands[-1]
                    operands.pop()

                    op2 = operands[-1]
                    operands.pop()

                    op = operators[-1]
                    operators.pop()
    
                    tmp = [op , op2 , op1]
                    operands.append(tmp)

                operators.pop()

            elif (not self.IsOperator(infix[i])):
                operands.append(infix[i])

            else:
                while (len(operators)!=0 and self.getPriority(infix[i]) <= self.getPriority(operators[-1])):
                    op1 = operands[-1]
                    operands.pop()
    
                    op2 = operands[-1]
                    operands.pop()
    
                    op = operators[-1]
                    operators.pop()
    
                    tmp = [op , op2 , op1]
                    operands.append(tmp)
                operators.append(infix[i])

        while (len(operators)!=0):
            op1 = operands[-1]
            operands.pop()
    
            op2 = operands[-1]
            operands.pop()
    
            op = operators[-1]
            operators.pop()
    
            tmp = [op , op2 , op1]
            operands.append(tmp)

        return self.flat(operands)


    def flat(self, lis):
        flatList = []
        # Iterate with outer list
        for element in lis:
            if type(element) is list:
                # Check if type is list than iterate through the sublist
                element = self.flat(element)
                for item in element:
                    flatList.append(item)
            else:
                flatList.append(element)
        return flatList

    def infixToPostfix(self, exp):
        stack = []
        output = []
        expression = self.Parser(exp)
        Priority = {'+':1, '-':1, '*':2, '/':2, '^':3}

        for item in expression:
            if not self.IsOperator(item):
                output.append(item)
            elif item=='(':
                stack.append('(')
            elif item==')':
                while stack and stack[-1]!= '(':
                    output.append(stack.pop())
                stack.pop()
            else: 
                while stack and stack[-1]!='(' and Priority[item]<=Priority[stack[-1]]:
                    output.append(stack.pop())
                stack.append(item)

        while stack:
            output.append(stack.pop())

        return output


    """
    def infix_to_prefix(self, expression):
        # Reverse the input list to make it easier to traverse
        expression.reverse()

        # Create an empty stack and output list
        stack = []
        output = []

        # Define operator precedence
        precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}

        # Loop through each element in the input list
        for elem in expression:
            if elem == '(':
                # Push left parenthesis onto stack
                stack.append(elem)
            elif elem == ')':
                # Pop elements from stack and add to output list until left parenthesis is reached
                while stack[-1] != '(':
                    output.append(stack.pop())
                stack.pop()
            elif elem in precedence:
                # While the stack is not empty and the top element has greater or equal precedence
                # than the current operator, pop the top element and add to output list
                while stack and stack[-1] != '(' and precedence[stack[-1]] >= precedence[elem]:
                    output.append(stack.pop())
                # Push the current operator onto stack
                stack.append(elem)
            else:
                # If the element is not an operator, add it to the output list
                output.append(elem)

        # Pop any remaining elements from the stack and add to the output list
        while stack:
            output.append(stack.pop())

        # Reverse the output list to get the prefix expression
        output.reverse()

        # Return the prefix expression
        return output
    """
    def treeToPostfix(self, Current = 0, result = None):
        
        CurrNode = self.Tree[Current]
        if result == None:
            result = []
        # check if CurrNode contains an operator
        if self.IsOperator(CurrNode.DataValue):
            if not self.IsOperator(self.Tree[CurrNode.LeftChild].DataValue):
                result.append(self.Tree[CurrNode.LeftChild].DataValue)
                print(result)
            if not self.IsOperator(self.Tree[CurrNode.RightChild].DataValue):
                result.append(self.Tree[CurrNode.RightChild].DataValue)
                print(result)
            # if LeftChild is an operator
            # traverse LeftChild subtree
            if self.IsOperator(self.Tree[CurrNode.LeftChild].DataValue):
                self.treeToPostfix(CurrNode.LeftChild, result)
                result.append(self.Tree[CurrNode.LeftChild].DataValue)
                print(result)
            # if RightChild is an operator
            # traverse RightChild subtree
            if self.IsOperator(self.Tree[CurrNode.RightChild].DataValue):
                self.treeToPostfix(CurrNode.RightChild, result)
                result.append(self.Tree[CurrNode.RightChild].DataValue)
                print(result)
        else:
            result.append(CurrNode.DataValue)
            print(result)
        return result

In [73]:
expressionTree = ExpressionTree()
expressionTree.MakeTree('2-27/3+567*2')
#print(expressionTree.infixToPrefix('2-27/3+567*2'))
print(expressionTree.infixToPostfix('2-27/3+567*2'))
# 3 + 56 * 2
# + - 2 / 27 3 * 567 2

['2', '-', '27', '/', '3', '+', '567', '*', '2']
['+', '-', '2', '/', '27', '3', '*', '567', '2']
Index:  0 DataValue:  +
Index:  1 DataValue:  -
Index:  2 DataValue:  2
Index:  3 DataValue:  /
Index:  4 DataValue:  27
Index:  5 DataValue:  3
Index:  6 DataValue:  *
Index:  7 DataValue:  567
Index:  8 DataValue:  2
Index:  9 DataValue:  
Index:  10 DataValue:  
Index:  11 DataValue:  
Index:  12 DataValue:  
Index:  13 DataValue:  
Index:  14 DataValue:  
Index:  15 DataValue:  
Index:  16 DataValue:  
Index:  17 DataValue:  
Index:  18 DataValue:  
Index:  19 DataValue:  
['2', '27', '3', '/', '-', '567', '2', '*', '+']


In [5]:
import re
regex_expression = "[\/\+\-\*\(\)]|[0-9][0-9][0-9]|[0-9][0-9]|[0-9]"
txt_list = re.findall(regex_expression, "(2*(3/1))+4")
print(expressionTree.calculate(txt_list))
txt_list = re.findall(regex_expression, "2+4")
print(expressionTree.calculate(txt_list))
txt_list = re.findall(regex_expression, "(2+4)-1")
print(expressionTree.calculate(txt_list))
txt_list = re.findall(regex_expression, "(10-(2+4)")
print(expressionTree.calculate(txt_list))

10
6
5
4


In [6]:
expression_string = expressionTree.Infix(Node.Expression)
print(expression_string)
#txt_list = re.findall(regex_expression, "(2-(27/3))+(567*2)")
#print(expressionTree.calculate(txt_list))
exp = re.findall(regex_expression, expression_string)
expressionTree.calculate(exp)

(2-(27/3))+(567*2)


1127

In [74]:
expressionTree.treeToPostfix()

['2']
['2', '27']
['2', '27', '3']
['2', '27', '3', '567']
['2', '27', '3', '567', '2']
['2', '27', '3', '567', '2', '*']
/
['2', '27', '3', '567', '2', '*', '/']
-
['2', '27', '3', '567', '2', '*', '/', '-']


['2', '27', '3', '567', '2', '*', '/', '-']